# AOC 2023

Welcome to the Advent of Code 2023 !

## Basic configuration



In [2]:
# help for aocd : https://pypi.org/project/advent-of-code-data/

#!pip install aocd

In [3]:
import os

# replace by your login session cookie
os.environ[
    "AOC_SESSION"
] = ""  # your login session cookie

In [4]:
from aocd import submit
from aocd.models import Puzzle

In [26]:
import numpy as np
from tqdm import tqdm
import json
import typing as typ
from collections import Counter, defaultdict, deque, ChainMap
import math
import itertools
import re
import regex
import string
import matplotlib.pyplot as plt
from functools import cache

## Day 13

https://adventofcode.com/2023/day/13

### Prepare input

In [ ]:
puzzle = Puzzle(year=2023, day=13)
content = puzzle.input_data.split("\n")
content

### Part 1

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
puzzle.answer_b = answ

## Day 12

https://adventofcode.com/2023/day/12

### Prepare input

In [27]:
puzzle = Puzzle(year=2023, day=12)
content =[]
for line in puzzle.input_data.split("\n"):
    row, groups = line.split()
    groups = tuple(map(int, groups.split(",")))
    content.append([row, groups])

In [28]:
#content

### Part 1

In [29]:
@cache
def count_arrangements(springs:str, groups:tuple[int], cur_group_len:int=0)->int: 
    #if we reached the end of row , check if the current arrangement is correct by checking groups
    if not springs: 
        return int(not groups) #if True, count +1 for arrangement
        
    count = 0
    start = ".#" if springs[0]=="?" else springs[0]
    for s in start: 
        if s == ".": 
            if cur_group_len: 
                if groups and groups[0] == cur_group_len: 
                    count += count_arrangements(springs[1:], groups[1:], 0) # reset group
            else: 
                count += count_arrangements(springs[1:], groups, 0)
        else:
            count += count_arrangements(springs[1:], groups, cur_group_len+1)
    return count

In [30]:
count_arrangements("???.###"+".",(1,1,3)) #putain de condition de fin

1

In [31]:
answ = sum(count_arrangements(s + ".", g) for s,g in content)

In [32]:
answ

7221

In [243]:
puzzle.answer_a = answ

That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two]


### Part 2

In [33]:
def unfold(springs:str, groups:tuple[int])->tuple[str,tuple[int]]: 
    return "?".join([springs]*5), groups*5

In [37]:
answ = sum(count_arrangements(s + ".", g) for s,g in [unfold(s,g) for s,g in content])
answ

7139671893722

In [36]:
puzzle.answer_b = answ

That's the right answer!  You are one gold star closer to restoring snow operations.You have completed Day 12! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


## Day 11

https://adventofcode.com/2023/day/11

### Prepare input

In [41]:
puzzle = Puzzle(year=2023, day=11)
content = [[0 if char=="." else 1 for char in line] for line in puzzle.input_data.split("\n")]

In [42]:
def get_empty(arr:typ.Union[list[list[int]], np.array])->list[int]: 
    return [idx for idx, row in enumerate(arr) if sum(row)==0]

empty_rows = get_empty(content)
empty_cols = get_empty(np.transpose(content))

In [154]:
empty_cols

[12, 37, 50, 68, 72, 79, 84, 88, 137]

In [155]:
galaxy_pos = [(x,y) for y in range(len(content)) for x in range(len(content[0])) if content[y][x] != 0]

### Part 1

In [192]:
def get_total_distance(expansion:int=1)->int:

    distances = []   
    for i in range(len(galaxy_pos) - 1):
        for j in range(i+1, len(galaxy_pos)):
            x1,y1 = galaxy_pos[i]
            x2,y2 = galaxy_pos[j]
            n_added_cols = sum(min(x1,x2) < col_idx < max(x1,x2) for col_idx in empty_cols) * expansion
            n_added_rows = sum(min(y1,y2) < row_idx < max(y1,y2) for row_idx in empty_rows) * expansion
            distances.append(abs(x1-x2) + abs(y1-y2) + n_added_cols + n_added_rows)

    return sum(distances)

In [193]:
answ=get_total_distance()
answ

9723824

In [194]:
puzzle.answer_a = answ

### Part 2

In [195]:
exp=1000000
answ=get_total_distance(exp-1)
answ

731244261352

In [191]:
puzzle.answer_b = answ

## Day 10

https://adventofcode.com/2023/day/10

### Prepare input

In [148]:
puzzle = Puzzle(year=2023, day=10)
content = puzzle.input_data.split("\n")
#content

In [110]:
for y,row in enumerate(content):
    for x,col in enumerate(row): 
        if col == "S":
            break
    else:
        continue
    break 
start = (x,y)

### Part 1

In [129]:
Pos = tuple[int,int]

def get_neighbours(pos:Pos)->list[Pos]: 
    x,y = pos
    def match_char(match_x:int,match_y:int)->list[bool]:
        # check in order : left right top bottom
        match content[match_y][match_x]: 
            case "|" : return [False,False,True,True]
            case "-" : return [True,True,False,False]
            case "L" : return [False,True,True,False]
            case "J" : return [True,False,True,False]
            case "7" : return [True,False,False,True]
            case "F" : return [False,True,False,True]
            case "." : return [False]*4 # ground has no pipe
            case "S" : return [True]*4 # start can be anything

    neighbours = []
    
    matches = match_char(x,y) 
    #check left
    if matches[0] and x > 0 and match_char(x-1,y)[1]:
        neighbours.append((x-1,y))
    #check right
    if matches[1] and x < len(content)-1 and match_char(x+1,y)[0]:
        neighbours.append((x+1,y))
    #check top
    if matches[2] and y > 0 and match_char(x,y-1)[3]:
        neighbours.append((x,y-1))
    #check bottom
    if matches[3] and y < len(content)-1 and match_char(x,y+1)[2]:
        neighbours.append((x,y+1))
        
    return neighbours

In [132]:
walk = [(start,None)]
steps = dict()
while walk: #while not empty 
    current, previous = walk.pop()
    if current == start and previous is not None:
        break 
    for next in get_neighbours(current): 
        if next != previous: 
            walk.append((next,current))
            steps[next]=current

In [134]:
path = [start]
while True:
    current = path[-1]
    previous = steps[current]
    if previous == start: 
        break
    path.append(previous)


In [138]:
answ = int(len(path)/2)
answ

6778

In [139]:
puzzle.answer_a = answ

### Part 2

In [146]:
count_inside = 0
for y, row in enumerate(content):
    n_crossing = 0
    for x, col in enumerate(row): 
        if (x,y) not in path:
            if n_crossing % 2 == 1: # if n_crossing is odd, 
                count_inside += 1 
        else: 
            if col in "|LJ":
                n_crossing += 1
count_inside

433

In [147]:
puzzle.answer_b = count_inside

## Day 9

https://adventofcode.com/2023/day/9

### Prepare input

In [93]:
puzzle = Puzzle(year=2023, day=9)
content = [list(map(int,line.split())) for line in puzzle.input_data.split("\n")]
#content

### Part 1

In [94]:
def get_diff(history: list[int])->list[list[int]]: 
    differences = [history]
    while not all(n==0 for n in differences[-1]): 
        hist = differences[-1]
        differences.append([y-x for x,y in zip(hist[:-1],hist[1:])])
    differences.reverse()
    return differences 
    
def get_next(diffs:list[list[int]])->int:
    diffs[0].append(0)
    for i in range(1,len(diffs)): 
        diffs[i].append(diffs[i-1][-1] + diffs[i][-1])
            
    return diffs[-1][-1]
        

In [95]:
answ = sum(get_next(get_diff(history)) for history in content)

In [96]:
answ

1995001648

In [25]:
puzzle.answer_a = answ

That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two]


### Part 2

In [97]:
def get_prev(diffs:list[list[int]])->int:
    diffs = [deque(d) for d in diffs]
    diffs[0].appendleft(0)
    for i in range(1,len(diffs)): 
        diffs[i].appendleft(diffs[i][0] - diffs[i-1][0])
    return diffs[-1][0]

In [98]:
answ = sum(get_prev(get_diff(history)) for history in content)

In [99]:
answ

988

In [100]:
puzzle.answer_b = answ

## Day 8

https://adventofcode.com/2023/day/8

### Prepare input

In [ ]:
puzzle = Puzzle(year=2023, day=8)
instructions, content = puzzle.input_data.split("\n\n")

In [ ]:
network = dict()
for line in content.split("\n"): 
    start, left, right = re.findall("\w+", line)
    network[start]={"L":left, "R":right}

In [ ]:
instructions, network

### Part 1

In [ ]:
def loop_on_condition(current, condition:typ.Callable[str, bool])->int: 
    for count, inst in enumerate(itertools.cycle(instructions)): 
            current = network[current][inst] 
            if condition(current): 
                return count + 1

In [ ]:
answ = loop_on_condition("AAA",lambda x : x == "ZZZ") 
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
starts = [key for key in network.keys() if key[-1]=="A"]
cycle_lengths = [loop_on_condition(start,lambda x : x[-1] == "Z") for start in starts]

In [ ]:
cycle_lengths

In [ ]:
answ = math.lcm(*cycle_lengths)
answ

In [ ]:
puzzle.answer_b = answ

## Day 7

https://adventofcode.com/2023/day/7

### Prepare input

In [84]:
puzzle = Puzzle(year=2023, day=7)
content = puzzle.input_data.split("\n")
content

['4KTJ4 575',
 '38T4K 449',
 '4T437 860',
 '55954 240',
 'Q8K89 150',
 '85838 21',
 '68668 464',
 'JQT77 737',
 '99497 155',
 'A444A 653',
 'AA6QQ 28',
 '4Q39T 332',
 '8Q579 835',
 '58JQ5 397',
 'KJ77A 271',
 '88Q8T 365',
 'AATAJ 30',
 '3T582 325',
 'Q8AQQ 321',
 'KA6J4 215',
 'KA322 403',
 '54225 219',
 '4ATA4 225',
 '4222A 543',
 '2J9KK 749',
 '477AT 561',
 '6J699 880',
 'J4348 655',
 'K4444 175',
 '775AA 567',
 'J8AK6 334',
 '93A58 307',
 'JT43J 872',
 'A6662 929',
 'Q74JK 779',
 'AQAQQ 827',
 '55557 902',
 '6JAA6 512',
 '676T2 763',
 'QA3KK 181',
 '99939 377',
 '43T44 473',
 'AK48A 968',
 '44464 897',
 '7A454 475',
 '6JJQ8 496',
 '5KAKK 419',
 '77AKT 802',
 'K4K4K 719',
 '33739 959',
 '325K2 534',
 '93979 751',
 'QK9AK 290',
 '8J887 79',
 '8QKQK 203',
 'Q657Q 328',
 '23647 869',
 '4568K 686',
 '49374 63',
 '88882 578',
 '94A9A 680',
 '5T5T5 759',
 '95JJ5 502',
 'Q66QQ 771',
 '6A886 845',
 '74777 982',
 '2K222 49',
 'TQTQT 704',
 'J7KK2 47',
 '98A5J 187',
 '66AA6 53',
 '57J68 443',


In [85]:
map_card = {"T":10,"J":11,"Q":12,"K":13,"A":14}
hands = [[[int(map_card.get(card, card)) for card in line.split()[0]], Counter(line.split()[0]), int(line.split()[1])] for line in content]
hands

[[[4, 13, 10, 11, 4], Counter({'4': 2, 'K': 1, 'T': 1, 'J': 1}), 575],
 [[3, 8, 10, 4, 13], Counter({'3': 1, '8': 1, 'T': 1, '4': 1, 'K': 1}), 449],
 [[4, 10, 4, 3, 7], Counter({'4': 2, 'T': 1, '3': 1, '7': 1}), 860],
 [[5, 5, 9, 5, 4], Counter({'5': 3, '9': 1, '4': 1}), 240],
 [[12, 8, 13, 8, 9], Counter({'8': 2, 'Q': 1, 'K': 1, '9': 1}), 150],
 [[8, 5, 8, 3, 8], Counter({'8': 3, '5': 1, '3': 1}), 21],
 [[6, 8, 6, 6, 8], Counter({'6': 3, '8': 2}), 464],
 [[11, 12, 10, 7, 7], Counter({'7': 2, 'J': 1, 'Q': 1, 'T': 1}), 737],
 [[9, 9, 4, 9, 7], Counter({'9': 3, '4': 1, '7': 1}), 155],
 [[14, 4, 4, 4, 14], Counter({'4': 3, 'A': 2}), 653],
 [[14, 14, 6, 12, 12], Counter({'A': 2, 'Q': 2, '6': 1}), 28],
 [[4, 12, 3, 9, 10], Counter({'4': 1, 'Q': 1, '3': 1, '9': 1, 'T': 1}), 332],
 [[8, 12, 5, 7, 9], Counter({'8': 1, 'Q': 1, '5': 1, '7': 1, '9': 1}), 835],
 [[5, 8, 11, 12, 5], Counter({'5': 2, '8': 1, 'J': 1, 'Q': 1}), 397],
 [[13, 11, 7, 7, 14], Counter({'7': 2, 'K': 1, 'J': 1, 'A': 1}), 271

### Part 1

In [86]:
def winnings(hands:list[list[typ.Any]])->int:

    def rank(x:int,y:int)->int: 
        return (x+1)*y
        
    def sorting_fnc(hand:typ.List[typ.Any])->typ.Tuple[typ.List[int],typ.List[int]]:
        return sorted(hand[1].values(), reverse=True), hand[0]

    return sum(rank(i,hand[2]) for i,hand in enumerate(sorted(hands, key=sorting_fnc)))

In [87]:
answ = winnings(hands)

In [88]:
answ

248179786

In [89]:
puzzle.answer_a = answ

### Part 2

In [ ]:
new_map_card = {"T":10,"J":0,"Q":12,"K":13,"A":14}
new_hands = []
for line in content: 
    hand, bid = line.split()
    hand_count = Counter(hand) 
    new_hand = [int(new_map_card.get(card, card)) for card in hand]

    j_count = hand_count["J"]
    if  j_count != 5: 
        hand_count["J"]=0
        mc = hand_count.most_common()[0]
        hand_count[mc[0]]= hand_count[mc[0]] + j_count
        
    new_hands.append([new_hand, hand_count, int(bid)])

In [ ]:
new_hands

In [ ]:
answ = winnings(new_hands)
answ

In [ ]:
puzzle.answer_b = answ

## Day 6

https://adventofcode.com/2023/day/6

### Prepare input

In [ ]:
puzzle = Puzzle(year=2023, day=6)
content = puzzle.input_data.split("\n")
content

In [ ]:
def parse_1(s:str)->typ.List[int]: 
    return list(map(int,re.findall("\d+", s)))
    
times = parse_1(content[0])
distances = parse_1(content[1])

### Part 1

In [ ]:
answ = math.prod(sum((t*(tot_time-t))>record_dist for t in range(tot_time)) for tot_time, record_dist in zip(times, distances))
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
def parse_2(s:str)->int:
    return int("".join(re.findall("\d+", s)))
    
time = parse_2(content[0])
dist = parse_2(content[1])

In [ ]:
answ = sum((t*(time-t))>dist for t in tqdm((range(time))))
answ

In [ ]:
puzzle.answer_b = answ

## Day 5

https://adventofcode.com/2023/day/5

### Prepare input

In [ ]:
puzzle = Puzzle(year=2023, day=5)
content =[line.split(":") for line in puzzle.input_data.split("\n\n")]
#content

In [ ]:
seeds = list(map(int, re.findall(r"\d+", content[0][1])))

In [ ]:
mappings = dict()
for line in content[1:]: 
    source_name, dest_name = re.findall(r"(\w+)-to-(\w+) map", line[0])[0]
    mappings[source_name] = {"dest": dest_name, 
                             "mapping": [list(map(int, re.findall(r"\d+", l))) for l in line[1].strip().split("\n")]}

In [ ]:
mappings

### Part 1

In [ ]:
def apply_maps(number:int,cur_source:str)->int: 
    if cur_source == "location":
        return number
    cur_map = mappings[cur_source]
    new_source = cur_map["dest"]
    for dest_start, source_start, range_len in cur_map["mapping"]: 
        if source_start <= number < source_start + range_len: 
            new_number = dest_start + number - source_start
            return apply_maps(new_number, new_source)
    return apply_maps(number, new_source)

In [ ]:
answ = min(apply_maps(seed_num, "seed") for seed_num in seeds)
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
reverse_mappings = dict()
for line in content[1:]: 
    source_name, dest_name = re.findall(r"(\w+)-to-(\w+) map", line[0])[0]
    reverse_mappings[dest_name] = {"source": source_name, 
                             "mapping": [list(map(int, re.findall(r"\d+", l))) for l in line[1].strip().split("\n")]}

In [ ]:
reverse_mappings

In [ ]:
def is_seed(number:int)->bool: 
    for start,range in zip(seeds[::2],seeds[1::2]):
        if start <= number < start+range: 
            return True
    return False

In [ ]:
def reverse_maps(number:int, cur_dest:str)->int: 
    if cur_dest == "seed": 
        return number
    cur_map = reverse_mappings[cur_dest]
    new_dest = cur_map["source"]
    for dest_start, source_start, range_len in cur_map["mapping"]: 
        if dest_start <= number < dest_start + range_len: 
            new_number = source_start + number - dest_start 
            return reverse_maps(new_number, new_dest)
    return reverse_maps(number, new_dest) 

In [ ]:
from itertools import count
for loc in tqdm(count()): 
    if is_seed(reverse_maps(loc, "location")):
        break
loc

In [ ]:
puzzle.answer_b = loc

## Day 4

https://adventofcode.com/2023/day/4

### Prepare input

In [ ]:
puzzle = Puzzle(year=2023, day=4)
content =[re.split(r":|\|", line) for line in puzzle.input_data.split("\n")]
content

### Part 1

In [ ]:
n_winnings = [(sum(x in list(map(int,ours.strip().split())) for x in list(map(int,winning.strip().split()))) ) for _,winning,ours in content]

In [ ]:
points = sum(2**(n-1) for n in n_winnings if n!=0)

In [ ]:
points

In [ ]:
puzzle.answer_a = points

### Part 2

In [ ]:
copies = {id+1 : 1 for id in range(len(content))}

In [ ]:
for cur_id, n_wins in enumerate(n_winnings): 
    for next_id in range(cur_id+1, cur_id+n_wins+1): 
        copies[next_id] += copies[cur_id]

In [ ]:
answ = sum(copies.values())
answ

In [ ]:
puzzle.answer_b = answ

## Day 3

https://adventofcode.com/2023/day/3

### Prepare input

In [75]:
puzzle = Puzzle(year=2023, day=3)
content = puzzle.input_data.split("\n")
content

['............830..743.......59..955.......663..........................................367...........895....899...............826...220......',
 '.......284.....*............*.....$...+.....*...377..................*.......419.............488...*.......*...................*..-....939..',
 '....%.........976..679.461.7..........350..33.........$.380...$...151.897..........295..#......*....105.....418.............481........&....',
 '...992.....#......=...../........701................508...*..578........................259...331.................795..945........79........',
 '.........868........................*.............................17*..........348................441*852........*.....-...........@.....922',
 '....................*200............311..63................452.......323.#778.*....674....................680......696...372.....*..........',
 '.......266.......209......589.....=......*...365.........7.*...233.............755....*......644...272........697..*....*.....68

### Part 1

In [78]:
def check_neighbours(array:list[str], row:int, col:int)->bool:

    def get_all_neighbours()->tuple[str,str,str,str,str,str,str,str]:
        top = array[row-1][col] if row != 0 else "."
        bottom = array[row+1][col] if row != len(array)-1 else "." 
        left = array[row][col-1] if col != 0 else "."
        right = array[row][col+1] if col != len(array[0])-1 else "."
        
        tl = array[row-1][col-1] if row != 0 and col != 0 else "."
        tr = array[row-1][col+1] if row != 0 and col != len(array[0])-1 else "."
        bl = array[row+1][col-1] if row != len(array)-1 and col != 0 else "."
        br = array[row+1][col+1] if row != len(array)-1 and col != len(array[0])-1 else "."
        
        return top, bottom, left, right, tl, tr, bl, br
    
    return any(n in string.punctuation.replace(".","") for n in get_all_neighbours())


In [79]:
parts = [(number, row) 
    for row, row_numbers in enumerate(re.finditer(r"\d+", line) for line in content) 
    for number in row_numbers
    if any(check_neighbours(content, row, col) for col in range(number.span()[0],number.span()[1]))
]

answ = sum(int(part[0].group()) for part in parts)

In [80]:
answ

521515

In [81]:
puzzle.answer_a = answ

### Part 2

In [72]:
Position = tuple[int,int]

def find_gears(array:list[str], row:int, col:int)->list(Position): 
    
    def get_neighbours_pos()->list[Position]: 
        neighbours = []
        if col != 0: 
            neighbours.append((row, col-1))
        if col != len(array[0])-1: 
            neighbours.append((row, col+1))
        if row != 0: 
            neighbours.append((row-1, col))
            if col != 0: 
                neighbours.append((row-1, col-1))
            if col != len(array[0])-1: 
                neighbours.append((row-1, col+1))
        if row != len(array)-1: 
            neighbours.append((row+1,col))
            if col != 0: 
                neighbours.append((row+1, col-1))
            if col != len(array[0])-1: 
                neighbours.append((row+1, col+1))

        return neighbours

    return [n for n in get_neighbours_pos() if array[n[0]][n[1]] == "*"]
                                  

In [ ]:
gears = defaultdict(list)
for part in parts: 
    number = part[0]
    row = part[1]
    gear_positions = set()
    for col in range(number.span()[0],number.span()[1]): 
        gear_positions.update(find_gears(content, row, col))
    for gear_pos in gear_positions: 
        gears[gear_pos].append(int(number.group()))

In [ ]:
answ = sum(math.prod(gear_parts) for gear, gear_parts in gears.items() if len(gear_parts)==2) 
answ 

In [ ]:
puzzle.answer_b = answ

## Day 2

https://adventofcode.com/2023/day/2

### Prepare input

In [66]:
puzzle = Puzzle(year=2023, day=2)

In [67]:
content = puzzle.input_data.split("\n")
content

['Game 1: 1 blue; 4 green, 5 blue; 11 red, 3 blue, 11 green; 1 red, 10 green, 4 blue; 17 red, 12 green, 7 blue; 3 blue, 19 green, 15 red',
 'Game 2: 17 red, 10 green; 3 blue, 17 red, 7 green; 10 green, 1 blue, 10 red; 7 green, 15 red, 1 blue; 7 green, 8 blue, 16 red; 18 red, 5 green, 3 blue',
 'Game 3: 10 blue, 3 green, 8 red; 15 green, 14 blue, 1 red; 8 blue, 11 red, 2 green; 5 red, 9 green, 6 blue',
 'Game 4: 1 red, 3 blue; 3 blue, 3 green, 1 red; 11 blue, 2 green; 2 green, 14 blue; 1 green, 7 blue; 11 blue, 5 green',
 'Game 5: 9 green, 5 red, 10 blue; 9 red, 4 blue, 12 green; 9 green, 6 blue, 14 red; 16 green, 8 red, 6 blue; 11 blue, 13 red, 1 green',
 'Game 6: 1 blue, 2 green, 16 red; 1 green, 19 red; 1 blue; 3 blue, 2 red, 1 green; 18 red, 2 blue, 1 green',
 'Game 7: 2 blue, 9 red, 5 green; 11 blue, 6 green, 4 red; 7 red, 3 green, 5 blue; 5 green, 11 blue, 7 red; 17 blue, 4 red, 3 green; 20 blue, 1 green, 2 red',
 'Game 8: 1 green, 6 red, 4 blue; 8 green, 4 blue, 2 red; 2 blue, 5 

In [ ]:
def get_max_colors(line:str)->dict[str,int]: 
    max_colors = {"red":0,"blue":0,"green":0}
    for number, color in re.findall(r"(\d+) (\w+)", line.split(": ")[1]): 
        max_colors[color] = max(max_colors[color], int(number))
    return max_colors
    

In [69]:
all_max_colors = [get_max_colors(line) for line in content]

### Part 1

In [ ]:
target = {"red":12,"green":13,"blue":14}

In [ ]:
answ = sum((id+1)*all(max_colors[key]<=target[key] for key in max_colors.keys()) for id, max_colors in enumerate(all_max_colors))
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
answ = sum(math.prod(max_colors.values()) for max_colors in all_max_colors)
answ

In [ ]:
puzzle.answer_b = answ

## Day 1

https://adventofcode.com/2023/day/1

### Prepare input

In [ ]:
puzzle = Puzzle(year=2023, day=1)

In [ ]:
content = puzzle.input_data.split("\n")

### Part 1

In [ ]:
answ = sum(int(f"{x[0]}{x[-1]}") for x in [re.findall(r"\d", line) for line in content])
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
txt_digits = "one, two, three, four, five, six, seven, eight, nine".split(", ")

In [ ]:
mapping = [{k:int(v),v:int(v)} for k,v in zip(txt_digits,string.digits[1:])]
mapping = dict(ChainMap(*mapping))

In [ ]:
numbers = [regex.findall(fr"\d|{'|'.join(txt_digits)}", line, overlapped=True) for line in content]

In [ ]:
answ = sum(mapping[x[0]]*10 + mapping[x[-1]] for x in numbers)
answ

In [ ]:
puzzle.answer_b = answ